# Some playing around with the data and sentiment


In [38]:
%matplotlib inline
import numpy as np
import pandas as pd
import re
from matplotlib import pyplot as plt
from itertools import islice
from string import punctuation
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from __future__ import division

dataset = pd.read_csv('../data/train.csv', encoding='utf-8').fillna('').drop(['id'],1)
my_stopwords = sorted (set (stopwords.words('english')))   # allow for easy adding of additional libraries



In [2]:
dataset.head(20)

,qid1,qid2,question1,question2,is_duplicate
0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [3]:
dataset.shape

(404290, 5)

A look at the unique questions:

In [4]:
distinct_question_ids = set(list(dataset['qid1'] + list(dataset['qid2'])))
print('Number of distinct questions: {0}'.format(len(distinct_question_ids)))

Number of distinct questions: 347345


In [5]:
columns = ['qid', 'question']
actual_questions = {}
for index, row in dataset.iterrows():
    if row['qid1'] not in actual_questions:
        actual_questions[row['qid1']] = row['question1']
    if row['qid2'] not in actual_questions:
        actual_questions[row['qid2']] = row['question2']


In [6]:
actual_questions_df = pd.DataFrame(list(actual_questions.iteritems()), columns=['qid', 'question'])

In [7]:
print 'Number unique questions (by ID): ', len(actual_questions)
print actual_questions_df.shape

Number unique questions (by ID):  537933
(537933, 2)


## Just for getting an idea, load the test set and see how many entries are in there:

In [8]:
testset = pd.read_csv('../data/test.csv')
print testset.shape

(2345796, 3)


2.3 mil question pairs. ok, that will need some comupting power, so keep in mind that runs won't be at the snap of a finger.

Training set: So rather than processing 404k question pairs, processing 538k uniques questions and then re-pairing them seems comutationally more efficient. Especially when considering the 2.3 mil question pairs in the test set. 

to-do: prep the test set for unique questions.

Ok, so let's use a subset of the actual_questions_df to play around with sentiment (just to keep processing time down.) I'll use a randomw split for 10% with seed 999 for this.

In [9]:
sample_df = actual_questions_df.sample(frac=0.1, random_state=999)
print sample_df.shape, ' expected: (53793, 2)'

(53793, 2)  expected: (53793, 2)


In [10]:
print sample_df.head(20)

           qid                                           question
321746  321747                       How did you prepare for MBA?
255670  255671                      Do I have to divorce my wife?
516117  516118   What are the best examples of good house design?
145912  145913       What will be the impact of GST on IT Sector?
134304  134305                 How can I be an Hollywood actress?
294295  294296              What could potentially replace Quora?
163150  163151  Where can I get a place to stay in goa for a l...
163010  163011  What is the current placement scenario for M.T...
362452  362453      How many employees does Quora currently have?
231268  231269  Why is it seen as racist to generalize the beh...
295059  295060  What are the themes explored in the book 'The ...
341174  341175  People who don't have any friends, how do you ...
209121  209122                     What does success mean to you?
532747  532748  Can Shia faith be described as an offshoot of ...
308748  30

Yep, looks alright. so let'd get to some preprocessing:


In [11]:
def remove_punctuation(s):
    s = ''.join([i for i in s if i not in frozenset(punctuation)])
    return s
def stemming(s):
    ps = PorterStemmer()
    return (' '.join(ps.stem(single) for single in s.split(' ')))
def cleaning(s):
    s = s.replace('-', ' ')                    # Make individual words out of words with "-"
    htmler = re.compile('<*?>')
    cleaned = re.sub(htmler, '', s)            # remove web / html notation
    cleaned = ' '.join([x for x in cleaned.split(' ') if x not in my_stopwords])
    cleaned = cleaned.replace('\n', ' ')       # remove line breaks
    cleaned = cleaned.replace('  ', ' ').replace('  ', ' ').replace('  ', ' ')   # reduce multiple blank spaces between words to single blank
    cleaned = cleaned.strip()                  # remove extra spaces before and after words
    return (cleaned)

In [18]:
sample_df['cleaned'] = sample_df['question'].apply(remove_punctuation)
sample_df['cleaned'] = sample_df['cleaned'].apply(cleaning)
sample_df['cleaned'] = sample_df['cleaned'].apply(stemming)

In [19]:
print sample_df.head(10)

           qid                                           question  \
321746  321747                       How did you prepare for MBA?   
255670  255671                      Do I have to divorce my wife?   
516117  516118   What are the best examples of good house design?   
145912  145913       What will be the impact of GST on IT Sector?   
134304  134305                 How can I be an Hollywood actress?   
294295  294296              What could potentially replace Quora?   
163150  163151  Where can I get a place to stay in goa for a l...   
163010  163011  What is the current placement scenario for M.T...   
362452  362453      How many employees does Quora currently have?   
231268  231269  Why is it seen as racist to generalize the beh...   

                                                  cleaned  
321746                                     How prepar MBA  
255670                                   Do I divorc wife  
516117                  What best exampl good hous design  


In [20]:
def number_words(s):
    num_words = len(s.split())
    return num_words

def number_chars(s):
    num_char = len(s)
    return num_char


In [21]:
sample_df['num_words'] = sample_df['cleaned'].apply(number_words)
sample_df['num_chars'] = sample_df['cleaned'].apply(number_chars)


In [22]:
print sample_df.head(10)

           qid                                           question  \
321746  321747                       How did you prepare for MBA?   
255670  255671                      Do I have to divorce my wife?   
516117  516118   What are the best examples of good house design?   
145912  145913       What will be the impact of GST on IT Sector?   
134304  134305                 How can I be an Hollywood actress?   
294295  294296              What could potentially replace Quora?   
163150  163151  Where can I get a place to stay in goa for a l...   
163010  163011  What is the current placement scenario for M.T...   
362452  362453      How many employees does Quora currently have?   
231268  231269  Why is it seen as racist to generalize the beh...   

                                                  cleaned  num_words  \
321746                                     How prepar MBA          3   
255670                                   Do I divorc wife          4   
516117                  

In [23]:
positive_words = open('../data/positive.txt').read()
positive_words_list = positive_words.split('\n')
negative_words = open('../data/negative.txt').read()
negative_words_list = negative_words.split('\n')

In [49]:
def sentiment(s):
    words = s.split(' ')
    length = len(words)
    sentiment_list = list()
    if (length // 2) < 7:
        max_len = (length // 2)
    else:
        max_len = 7
    for i in range(1, max_len + 1):
        sentiment_sub_list = list()
        for j in range(len(words) - i + 1):
            positive_counter = 0
            negative_counter = 0
            words_rest = words[j:j+i]
            for word in words_rest:
                if word in positive_words_list:
                    positive_counter += 1
                if word in negative_words_list:
                    negative_counter += 1
            #print i, ', ', j, ', ', positive_counter, ', -', negative_counter, ', ', (float(positive_counter) / i) - (float(negative_counter) / i)
            sentiment_sub_list.append((float(positive_counter) / i) - (float(negative_counter) / i))
        sentiment_list.append([i, sentiment_sub_list])
    return sentiment_list
                

In [ ]:
sample_df['sentiment'] = sample_df['cleaned'].apply(sentiment)

In [26]:
print sample_df.head(10)

           qid                                           question  \
321746  321747                       How did you prepare for MBA?   
255670  255671                      Do I have to divorce my wife?   
516117  516118   What are the best examples of good house design?   
145912  145913       What will be the impact of GST on IT Sector?   
134304  134305                 How can I be an Hollywood actress?   
294295  294296              What could potentially replace Quora?   
163150  163151  Where can I get a place to stay in goa for a l...   
163010  163011  What is the current placement scenario for M.T...   
362452  362453      How many employees does Quora currently have?   
231268  231269  Why is it seen as racist to generalize the beh...   

                                                  cleaned  num_words  \
321746                                     How prepar MBA          3   
255670                                   Do I divorc wife          4   
516117                  

In [39]:
print sample_df.ix[231268, 'sentiment']

[[1, [0.0, 0.0, -1.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0]], [2, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0]], [3, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0]], [4, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.0]], [5, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.2, 0.2, 0.2, 0.4, 0.2, 0.2, 0.2, 0.2]], [6, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666

In [43]:
print sentiment(sample_df.ix[231268, 'cleaned'])

1 ,  0 ,  0 , - 0 ,  0.0
1 ,  1 ,  0 , - 0 ,  0.0
1 ,  2 ,  0 , - 1 ,  -1.0
1 ,  3 ,  0 , - 0 ,  0.0
1 ,  4 ,  0 , - 0 ,  0.0
1 ,  5 ,  0 , - 1 ,  -1.0
1 ,  6 ,  0 , - 0 ,  0.0
1 ,  7 ,  0 , - 0 ,  0.0
1 ,  8 ,  0 , - 0 ,  0.0
1 ,  9 ,  0 , - 1 ,  -1.0
1 ,  10 ,  0 , - 0 ,  0.0
1 ,  11 ,  0 , - 0 ,  0.0
1 ,  12 ,  0 , - 0 ,  0.0
1 ,  13 ,  0 , - 0 ,  0.0
1 ,  14 ,  0 , - 1 ,  -1.0
1 ,  15 ,  0 , - 0 ,  0.0
1 ,  16 ,  0 , - 0 ,  0.0
1 ,  17 ,  0 , - 0 ,  0.0
1 ,  18 ,  0 , - 1 ,  -1.0
1 ,  19 ,  0 , - 1 ,  -1.0
1 ,  20 ,  0 , - 0 ,  0.0
1 ,  21 ,  1 , - 0 ,  1.0
1 ,  22 ,  0 , - 0 ,  0.0
1 ,  23 ,  0 , - 0 ,  0.0
1 ,  24 ,  0 , - 0 ,  0.0
1 ,  25 ,  1 , - 0 ,  1.0
1 ,  26 ,  0 , - 0 ,  0.0
1 ,  27 ,  0 , - 0 ,  0.0
1 ,  28 ,  0 , - 0 ,  0.0
1 ,  29 ,  0 , - 1 ,  -1.0
2 ,  0 ,  0 , - 0 ,  0.0
2 ,  1 ,  0 , - 1 ,  -0.5
2 ,  2 ,  0 , - 1 ,  -0.5
2 ,  3 ,  0 , - 0 ,  0.0
2 ,  4 ,  0 , - 1 ,  -0.5
2 ,  5 ,  0 , - 1 ,  -0.5
2 ,  6 ,  0 , - 0 ,  0.0
2 ,  7 ,  0 , - 0 ,  0.0
2 ,  8 ,  0 , - 1 ,